In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from imblearn.ensemble import RUSBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from time import time
import random
import seaborn as sns
from statistics import mode

In [ ]:
ignore_session_threashold = 0.50
data_status = 'old_'
data_source = 'rwt'
tcr_subject = [7, 112, 113, 121, 75, 107, 79, 82, 118, 76, 115, 117, 119, 120, 105, 78, 124]
rwt_subject = [7, 112, 113, 114, 75, 107, 79, 82, 118, 76, 115, 117, 119, 120]
new_tcr_trial = [123, 124]
subject = None
if data_status == "old_":
    if data_source == 'tcr':
        subject = tcr_subject
    else:
        subject = rwt_subject
else:
    subject = new_tcr_trial
first_chopped_off = 600 * 0.3
last_chopped_off = 600 * 0

Random_Forest_predicted_y = []
RBF_SVM_predicted_y = []


GradientBoost = {}
NearestNeighbor = {}
AdaBoost = {}
RandomForest = {}
LinearSVM = {}
RBFSVM = {}
DecisionTree = {}
RUSBoost = {}
LDA = {}

data_left_for_subjects = {} # records the percentage of data being used for each subject

time_continuity_subject = {} # records the time of time_continuity_algorithm for each subject

subject_accuracy_random_forest_dict = {} # records the accuracy of random forest for each subject

subject_accuracy_dict = {} # records the accuracy of time_continuity_algorithm for each subject

subject_preprocess_record = {} # records the number of sessions and folds left for each subjects

subject_prediction = {}

subject_unknown_percentage = {}

time_classifier = {} # records the time it takes for each classifier to execute the 7-fold cross-validation
# define models to train
names = [  
#         'GradientBoostingRegressor',
#         'Nearest Neighbors',
#         'AdaBoostClassifier',
        'RandomForest',
#         "Linear SVM",
        "RBF SVM",
#         "Decision Tree",
#         'RUSBoost',
#         'Shrinkage LDA',
        ]

# build classifiers
classifiers = [
#             GradientBoostingRegressor(random_state=1),
#             KNeighborsClassifier(n_neighbors=5),
#             AdaBoostClassifier(n_estimators=400, learning_rate = 0.6),
            RandomForestClassifier(n_estimators=300, max_features = "sqrt", oob_score = True),
#             SVC(kernel="linear", C=0.025),
            SVC(gamma=2, C=1),
#             DecisionTreeClassifier(),
#             RUSBoostClassifier(n_estimators = 200, random_state=1),
#             LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto'),
              ]

dicts_records = [
#                  GradientBoost, 
#                  NearestNeighbor,
#                  AdaBoost, 
                 RandomForest, 
#                  LinearSVM,
                 RBFSVM,
#                  DecisionTree,
#                  RUSBoost, 
#                  LDA
                ]


def check_removed_index(name, removed_dict, index_to_be_removed, lowerBound, upperBound):
    if name not in removed_dict:
        removed_dict[name] = 0
    lst = [i for i in index_to_be_removed if i >= lowerBound and i < upperBound]
    removed_dict[name] = removed_dict[name] + len(lst)

def calculate_accuracy(y_actual, y_predict):
    count = 0
    for i in range(len(y_actual)):
        if y_actual[i] == y_predict[i]:
            count = count + 1
    return count / float(len(y_actual))

def check_plateau(dataFrame, current_index):
    for i in range(current_index + 1, current_index + 14):
        if i >= len(dataFrame):
            return True
        lst = move_data.iloc[current_index, :].tolist()
        if sum(lst) != 0:
            return False
    return True


def random_output(first, second):
    random_num = random.random()
    if random_num <= 0.5:
        return first
    else:
        return second

print(len(subject))

tasks_dict = {} # array of dictionaries
for i in range(30):
    key_name = "task" + str(i+1)
#     temp_dict = {}
#     temp_dict[key_name] = []
#     tasks_dict.append(temp_dict)
    tasks_dict[key_name] = []

print(tasks_dict)

def most_common(List):
    return(mode(List))

def accuracy_divide(numerator, denominator):
    if denominator == 0:
        return 0
    return numerator / denominator
# subject_2_RF = []
# subject_3_RF = []

In [ ]:
# for i in range(1, len(subject) + 1):
subject_id_start = 1
subject_id_end = 5
for i in range(subject_id_start, subject_id_end):
    tasks_dict.clear() # array of dictionaries
    for task_rank in range(30):
        key_name = "task" + str(task_rank+1)
        tasks_dict[key_name] = []
    subject_id = i
    print()
    print("checking subject",subject_id)
    print()
    frame = []
    for session in range(1,7):
        data_one = pd.read_csv('data_preprocess/'+data_status + data_source+'_plateau_removed_data/'+data_source+"_subject_"+str(subject_id)+"_session_"+str(session)+".csv",
                      header = None)
        zeros = [0] * 20

        if len(data_one) <= 3000:
            data_one.loc[len(data_one)] = zeros
        data_one = data_one.iloc[0:3000]
        temp_frame = []
        for i in range(5):
            temp = data_one.iloc[int(i * 600 + first_chopped_off) : int((i + 1) * 600 - last_chopped_off)]
            temp_frame.append(temp)
        data_one = pd.concat(temp_frame)
        frame.append(data_one)
    data = pd.concat(frame)
    data.reset_index(drop=True, inplace=True)
    
    # check plateau (noise)
    index_to_be_removed = []
    for session in range(0,6):
        temp_data = data.iloc[session * 2100 : (session + 1) * 2100]
        for move in range(0,5):
            move_data = temp_data.iloc[move * 420 : (move + 1) * 420]
#             i = 0
#             while i < len(move_data):
#                 lst = move_data.iloc[i, :].tolist()
#                 temp_lst = []
#                 if sum(lst) == 0:
#                     if check_plateau(move_data, i):
#                         end = i + 14
#                         if end >= len(move_data):
#                             end = len(move_data)
#                         temp_lst = [j for j in range(session * 2100 + move * 420 + i, session * 2100 + move * 420 + end)]
#                         i = i + 14
#                     else:
#                         i = i + 1
#                 if len(temp_lst) > 0:
#                     index_to_be_removed.extend(temp_lst)
            for i in range(len(move_data)):
                lst = move_data.iloc[i, :].tolist()
                if sum(lst) == 0:
                    index_to_be_removed.append(session * 2100 + move * 420 + i)
    print("number of rows to be removed is", len(index_to_be_removed))
    
    # add labels
    ones = [1] * int(600 * 0.7)
    twos = [2] * int(600 * 0.7)
    threes = [3] * int(600 * 0.7)
    fours = [4] * int(600 * 0.7)
    fives = [5] * int(600 * 0.7)
    len(fives)
    session1 = ones + twos + threes + fours + fives
    session2 = fours + ones + twos + threes + fives
    session3 = ones + fours + threes + twos + fives
    session4 = ones + twos + threes + fours + fives
    session5 = twos + ones + threes + fives + fours
    session6 = ones + twos + fours + threes + fives
    session_all = session1 + session2 + session3 + session4 + session5 + session6
    data["ground_truth"] = session_all
    
    #check if the subject should be kept
    percentage_removed_total = (int(18000 * 0.7) - len(index_to_be_removed)) / 18000.0
    print("percentage of data left for subject", subject_id, "is", percentage_removed_total)
    if percentage_removed_total < 0.35:
        print("the subject", subject_id, "should be removed and will be ignored")
        continue
    if str(subject_id) not in subject_unknown_percentage:
        subject_unknown_percentage[str(subject_id)] = {}
    subject_unknown_percentage[str(subject_id)]["known"] = percentage_removed_total 
    subject_preprocess_record[str(subject_id)] = {}
        
    # checks each six session:
    print("check session for subject", subject_id)
    session_list = [i for i in range(0, 6)]
    for session in range(0, 6):
        session_lowerbound = 2100 * session
        session_upperbound = 2100 * (session + 1)
        to_be_removed = [i for i in index_to_be_removed if i >= session_lowerbound and i < session_upperbound]
        print("Number of rows to be removed for session", (session + 1) , "is", len(to_be_removed))
        percentage_remained = (2100 - (len(to_be_removed))) / 3000.0
        print("percent of rows left in sesssion", (session + 1), "is", percentage_remained)
        if percentage_remained < 0.35:
            session_list.remove(session)
            print("session " + str(session + 1) + " should be removed and will be ignored")
            print()
        print()
    if len(session_list) == 0:
        print("all sessions are ignored. Continue to next person")
        continue
    
    subject_preprocess_record[str(subject_id)]["session_remained"] = len(session_list)
    
    not_session_list = [i for i in range(0, 6) if i not in session_list]
    for i in not_session_list:
        for j in range(i*5, (i+1)*5):
            print("j is",j)
            tasks_dict["task"+str(j+1)].extend([-1]*420)
#     print(tasks_dict)
#     print("Test")
#     print(tasks_dict["task5"].count(-1))
#     print("Test")
    # cut 7 folds
    test1data = []
    test2data = []
    test3data = []
    test4data = []
    test5data = []
    test6data = []
    test7data = []
    removed_dict = {}
    fold_names = ["fold1", "fold2", "fold3", "fold4", "fold5", "fold6", "fold7"]
    fold_names_dict = {}
    for i in range(len(fold_names)):
        name = fold_names[i]
        fold_names_dict[name] = i + 1
#     print("the fold_names_dict is", fold_names_dict)
    # for each move (420 lines), split the data into seven folds
    # at the same time, record the number of lines being that would be omited
    # remove folds that have less than 33.33% data remained
    #each fold should have at most 60 * 30 = 1800 (originally 2571.4)
    move_lst = []
    for ele in session_list:
        temp = [i for i in range(ele*5, (ele + 1)*5)]
        move_lst.extend(temp)
    for i in move_lst:
        lowerBound = i * 420
        test1data.append(data.iloc[lowerBound : lowerBound + 60])
        check_removed_index("fold1", removed_dict, index_to_be_removed, lowerBound, lowerBound + 60)
        test2data.append(data.iloc[lowerBound + 60 : lowerBound + 120])
        check_removed_index("fold2", removed_dict, index_to_be_removed, lowerBound + 60, lowerBound + 120)
        test3data.append(data.iloc[lowerBound + 120 : lowerBound + 180])
        check_removed_index("fold3", removed_dict, index_to_be_removed, lowerBound + 120, lowerBound + 180)
        test4data.append(data.iloc[lowerBound + 180 : lowerBound + 240])
        check_removed_index("fold4", removed_dict, index_to_be_removed, lowerBound + 180, lowerBound + 240)
        test5data.append(data.iloc[lowerBound + 240 : lowerBound + 300])
        check_removed_index("fold5", removed_dict, index_to_be_removed, lowerBound + 240, lowerBound + 300)
        test6data.append(data.iloc[lowerBound + 300 : lowerBound + 360])
        check_removed_index("fold6", removed_dict, index_to_be_removed, lowerBound + 300, lowerBound + 360)
        test7data.append(data.iloc[lowerBound + 360 : lowerBound + 420])
        check_removed_index("fold7", removed_dict, index_to_be_removed, lowerBound + 360, lowerBound + 420)

    folds_list = [test1data, test2data, test3data, test4data, test5data, test6data, test7data]
    # check folds percentages
    for name in fold_names:
        removed_num = removed_dict[name]
        remained_percentage = (((2100 * len(session_list))/ 7.0) - removed_num) / ((3000 * len(session_list)) / 7.0)
        print()
        print("the " + name + " has", remained_percentage, "left")
        print()
        if remained_percentage < 0.35:
            print(name + " should be removed for subject", subject_id)
            idx = fold_names.index(name)
            print("the index to be removed is", idx)
            del folds_list[idx]
            fold_names.remove(name)
    folds = []
    print(removed_dict)
    print(sum(removed_dict.values()))
    for fold in folds_list:
        data = pd.concat(fold)
        print(data.shape)
        folds.append(data)
    if len(folds) == 0:
        print("all folds are ignored. Continue to next person")
        continue
    
    folds_predicted_lst = []
    subject_preprocess_record[str(subject_id)]["folds_remained"] = len(folds)
    
    t_start = time()
    subject_prediction[str(subject_id)] = {}
    models = zip(names, classifiers, dicts_records)
    Random_Forest_predicted_y.clear()
    RBF_SVM_predicted_y.clear()
    for name, classifier, dicts_record in models:
        for key in tasks_dict:
            if len(tasks_dict[key]) > 0 and tasks_dict[key].count(-1) == 420:
                continue
            else:
                tasks_dict[key] = []
        
        print("DEBUG after clearing tasks_dict")
        print(tasks_dict)
#         tasks_dict.clear() # array of dictionaries
#         for i in range(30):
#             key_name = "task" + str(i+1)
#         #     temp_dict = {}
#         #     temp_dict[key_name] = []
#         #     tasks_dict.append(temp_dict)
#             tasks_dict[key_name] = []
        folds_predicted_lst.clear()
#         if name == "RandomForest":
#             folds_predicted_lst = Random_Forest_predicted_y
#         if name == "RBF SVM":
#             folds_predicted_lst = RBF_SVM_predicted_y
        accuracy = 0
        t0 = time()
        execute_counter = 1
        for i in range(len(folds_list)):
            folds.append(folds.pop(0)) # move the first fold to the last, and iterate it 7 times
            data = pd.concat(folds[:-1])
            X = data.iloc[:, :-1]
            y = data.iloc[:, -1]
            clf = classifier
            clf.fit(X,y)
            data_test = folds[-1]
            X_test = data_test.iloc[:, :-1]
            y_test = data_test.iloc[:, -1]
            y_predict = []
            if name == "GradientBoostingRegressor":
                y_predict = clf.predict(X_test)
                accuracy = accuracy + clf.score(X_test, y_test)
            else:
                y_predict = clf.predict(X_test)
                folds_predicted_lst.append(y_predict)
                print("executed",execute_counter,"times")
                accuracy = accuracy + calculate_accuracy(y_test.tolist(), y_predict)
            execute_counter += 1
        t1 = time()
        time_elapsed = t1 - t0
        print()
        print("The time it takes to run " + name + " is", time_elapsed)
#         if name == 'RandomForest':
#             clf_tmp = classifier
#             clf_tmp.fit()
#             Random_Forest_predicted_y.extend(y_temp_predict)
#         if name == 'RBF SVM':
#             RBF_SVM_predicted_y.extend(y_predict)
        if name not in time_classifier:
            time_classifier[name] = 0
        time_classifier[name] = time_classifier[name] + time_elapsed
        accuracy = accuracy / float(len(folds_list))
        subject_prediction[str(subject_id)][name] = {}
        subject_prediction[str(subject_id)][name]["acutual_y"] = y_test
        subject_prediction[str(subject_id)][name]["predicted_y"] = y_predict
        dicts_record[str(subject_id)] = accuracy
        print("The accuracy of subject", subject_id, "is", accuracy, "with the model " + name)

        print()
        print("length of the folds_predicted_lst for " + name + " is", len(folds_predicted_lst))
        print()

        fold_new_names = ["fold1", "fold2", "fold3", "fold4", "fold5", "fold6", "fold7"]
#         print()
#         print("folds_predicted_lst is", folds_predicted_lst)
#         print()
        folds_predicted_lst_counter = 0
        for fold_name in fold_new_names:
            if fold_name in fold_names:
                print(fold_name + " is in!")
    #             print("testing name is" + name + " \n")
                task_counter = 0
                temp_lst = folds_predicted_lst[folds_predicted_lst_counter]
                for key in tasks_dict:
    #                 print("testing key is " + key + " \n")
    #                 print(tasks_dict[key])
                    if len(tasks_dict[key]) > 0 and tasks_dict[key].count(-1) == 420:
                        print(key + " has been ignored, probably due to the ignored session of the subject", subject_id)
                        continue
                    tasks_dict[key].extend(temp_lst[task_counter * 60: (task_counter + 1)*60])
#                     print("just added 60 elements not ignored")
                    task_counter = task_counter + 1
                folds_predicted_lst_counter += 1
            else:
                print(fold_name + " is not in!")
                for key in tasks_dict:
                    if len(tasks_dict[key]) > 0 and tasks_dict[key].count(-1) == 420:
                        print(key + " has been ignored, probably due to the ignored session of the subject", subject_id)
                        continue
                    tasks_dict[key].extend([-1]*60)
#                     print("just added 60 elements TTTTTT ignored")
            

        print()
        print("Printing tasks_dict")
        count = 1
        for key in tasks_dict:
            print(str(count) + " " + str(len(tasks_dict[key])))
            count += 1
        print()


        if name == "RandomForest":
            for key in tasks_dict:
                Random_Forest_predicted_y.extend(tasks_dict[key])

            print()
            print("The predicted_y length of the "+ name + " is", len(Random_Forest_predicted_y))
            print()
        if name == "RBF SVM":
            for key in tasks_dict:
                RBF_SVM_predicted_y.extend(tasks_dict[key])

            print()
            print("The predicted_y length of the "+ name + " is", len(RBF_SVM_predicted_y))
            print()       
        
        
        print("="*20)
        print()

    # Time Majority Voting
    task_ground_truth = 1
    interval_lst = [(0, 419), (420, 839), (840, 1259), (1260, 1679), (1680, 2099)]

    table_data = defaultdict(list)
    print(subject_id)
    table_data = defaultdict(list)
    task_lst = []
    task_lst.append([1,2,1,1,2,1]) # task 1
    task_lst.append([2,3,4,2,1,2]) # task 2
    task_lst.append([3,4,3,3,3,4]) # task 3
    task_lst.append([4,1,2,4,5,3]) # task 4
    task_lst.append([5,5,5,5,4,5]) # task 5
    tasks_idx = [[],[],[],[],[]]
    should_be_ignored_lists = [[],[],[],[],[]]

    task_count = 1
    subject_accuracy_before_numerator = 0
    subject_accuracy_before_denominator = 0
    subject_accuracy_numerator = 0
    subject_accuracy_denominator = 0
    
    data_used = 0
    tmv_result_dict = {}
    for task_num, task_idx, ignored_lst in zip(task_lst, tasks_idx, should_be_ignored_lists):
        
        tmv_result_dict[str(task_count)] = []
        for i in range(len(task_num)):
            starting = 2100 * i
            pos = task_num[i]
            interval = interval_lst[pos - 1]
            idx_first = starting + interval[0]
            idx_second = starting + interval[1]
            task_idx.append((idx_first, idx_second))

        print("Task " + str(task_count) + " tasks indices")
        print(task_idx)
        print()

        session_count = 1
        i = 0
        length = len(task_idx)
        session_map = {}

        while i < length:
            
            interval = task_idx[i]
            session_map[interval] = session_count
            session = Random_Forest_predicted_y[interval[0] : interval[1]+1]
#             print("Testing session length is", len(session))
            session = [i for i in session if i != -1]
#             print("Testing after-session length is", len(session))
            matched_total = len([i for i in session if i == task_ground_truth])
            if(len(session) == 0):
                session.append(10)
            ground_truth_percentage = matched_total / len(session)
            print("session " + str(session_count) + " has percentage to the ground truth: " + str(ground_truth_percentage))
            print()
            if ground_truth_percentage < ignore_session_threashold:
                print("session " + str(session_count) + " should be ignored for the task_" + str(task_count))
                print()
                ignored_lst.append(interval)
            i += 1
            session_count += 1

        print("new task_" + str(task_count) + "_idx is", task_idx)
        print()
        print(session_map)
        print()
        print("Ignored intervals:", ignored_lst)
        print()

        total_accuracy_before_numerator = 0
        total_accuracy_before_denominator = 0
        total_accuracy_new_numerator = 0
        total_accuracy_new_denominator = 0
        task_tmp_before = []
        task_tmp_new = []
        for ele in task_idx: 
            if ele in ignored_lst:
                task_tmp_before.append(-1)
                task_tmp_new.append(-1)
                tmv_result_dict[str(task_count)].extend([-1]*420)
                continue
            session_num = session_map[ele]
            RF_y = Random_Forest_predicted_y[ele[0] : ele[1] + 1]
            RF_y_copy = Random_Forest_predicted_y[ele[0] : ele[1] + 1]
#             print("RF_y original is", len(RF_y))
            RBF_y = RBF_SVM_predicted_y[ele[0] : ele[1] + 1]
            RBF_y_copy = RBF_SVM_predicted_y[ele[0] : ele[1] + 1]
            RF_y = [i for i in RF_y if i != -1]
            data_used += len(RF_y)
#             print("RF_y remained is", len(RF_y))
            print()
            RBF_y = [j for j in RBF_y if j != -1]
#             print("RBF_y remained is", len(RBF_y))
#             print()
            majority_prediction = most_common(RF_y)
#             print("majority_prediction of", session_num, "is", majority_prediction)
            print()
            y_predict = []
            for j in range(len(RF_y_copy)):
                if RF_y_copy[j] == RBF_y_copy[j]:
                    tmv_result_dict[str(task_count)].append(RF_y_copy[j])
                else:
                    tmv_result_dict[str(task_count)].append(majority_prediction)
            for i in range(len(RF_y)):
                if RF_y[i] == RBF_y[i]:
                    y_predict.append(RF_y[i])
                else:
                    y_predict.append(majority_prediction)
            before_matched_predict = [j for j in RF_y if j == task_ground_truth]
            before_accuracy = len(before_matched_predict) / len(RF_y)
            task_tmp_before.append(before_accuracy)
            total_accuracy_before_numerator += len(before_matched_predict)
            total_accuracy_before_denominator += len(RF_y)
            print("session", session_num, "had the prediction accuracy before", before_accuracy)
            print()
            matched_predict = [j for j in y_predict if j == task_ground_truth]
            accuracy = len(matched_predict) / len(RF_y)
            task_tmp_new.append(accuracy)
            total_accuracy_new_numerator += len(matched_predict)
            total_accuracy_new_denominator += len(RF_y)
            print("session", session_num, "has the prediction accuracy", accuracy)

        total_accuracy_before = accuracy_divide(total_accuracy_before_numerator, total_accuracy_before_denominator)
        subject_accuracy_before_numerator += total_accuracy_before_numerator
        subject_accuracy_before_denominator += total_accuracy_before_denominator

        total_accuracy_new = accuracy_divide(total_accuracy_new_numerator, total_accuracy_new_denominator)
#         total_accuracy_new = total_accuracy_new_numerator / total_accuracy_new_denominator
#         print("DEBUG!")
#         print("subject ", subject_id)
#         print("total_accuracy_new_numerator", total_accuracy_new_numerator)
#         print("total_accuracy_new_denominator", total_accuracy_new_denominator)
        subject_accuracy_numerator += total_accuracy_new_numerator
        subject_accuracy_denominator += total_accuracy_new_denominator
#         print("subject_accuracy_numerator", subject_accuracy_numerator)
#         print("subject_accuracy_denominator", subject_accuracy_denominator)

        task_tmp_before.append(total_accuracy_before)
        task_tmp_new.append(total_accuracy_new)
        key_name_before = "task"+str(task_count)+"before"
        key_name_new = "task"+str(task_count)+"new"
        table_data[key_name_before] = task_tmp_before
        table_data[key_name_new] = task_tmp_new
        task_count = task_count + 1
        task_ground_truth += 1
        print("="*30)
        print()

    t_end = time()
    time_elapsed_TC = t_end - t_start 
    time_continuity_subject[subject_id] = time_elapsed_TC

    subject_accuracy_before = accuracy_divide(subject_accuracy_before_numerator, subject_accuracy_before_denominator)
#     subject_accuracy_before = subject_accuracy_before_numerator / subject_accuracy_before_denominator
    subject_accuracy_random_forest_dict[subject_id] = subject_accuracy_before
    print()
    print(str(subject_id) + "'s random forest has accuracy of " + str(subject_accuracy_before))
    print()
    subject_accuracy = accuracy_divide(subject_accuracy_numerator, subject_accuracy_denominator)
#     subject_accuracy = subject_accuracy_numerator / subject_accuracy_denominator
    subject_accuracy_dict[subject_id] = subject_accuracy
    print()
    print(str(subject_id) + "'s time continuity algorithm has accuracy of " + str(subject_accuracy))
    print()
    print("Table data currently is")
    print(table_data)

    for key in table_data:
        lst = table_data[key]
        new_ls = [round(i, 3) for i in lst]
        table_data[key] = new_ls

    idx_lst = []
    for i in range(1, 7):
        idx_lst.append("session"+str(i))
    idx_lst.append("accuracy")
    df = pd.DataFrame(table_data, index=idx_lst)
    df.to_csv("Time_continuity_results/"+data_source+"_subject_"+str(subject_id)+"_all_tasks.csv")

    print()
    print(str(subject_id) + " spent " + str(time_elapsed_TC) + " execute Time_continuity")
    data_left_for_subjects[subject_id] = data_used / 18000.0

In [ ]:
print("percentage left for all subjects", data_left_for_subjects)

In [ ]:
print()
print("Time continuity subject dictionary is", time_continuity_subject)
print()
print("subject accuracy_RF", subject_accuracy_random_forest_dict)
print()
print("subject accuracy_TC", subject_accuracy_dict)


# subject_id_order = [10, 12, 13, 15, 8, 17, 14, 2, 3, 5, 9, 1]
# subject_id_order = [1,2,3,4]
subject_id_order = [z for z in range(subject_id_start, subject_id_end)] 
subject_id_order.sort(key = lambda x : subject_accuracy_random_forest_dict[x], reverse=True)
plt.figure(figsize=(10, 5))
RF_predict = []
New_predict = []
percentage_left_subjects = []
time_data = []
for num in subject_id_order:
    RF_predict.append(subject_accuracy_random_forest_dict[num])
    New_predict.append(subject_accuracy_dict[num])
    percentage_left_subjects.append(data_left_for_subjects[num])
    time_data.append(time_continuity_subject[num])
    
RF_Phase2_accuracy = sum(RF_predict) / len(RF_predict)
TMV_accuracy = sum(New_predict) / len(New_predict)
print("="*20)
print("RF_Phase2_accuracy", RF_Phase2_accuracy)
print("TMV_accuracy", TMV_accuracy)
print("="*20)
print()
print("len of the time data", len(time_data))
X_axis = np.arange(len(subject_id_order))
plt.bar(X_axis - 0.2, RF_predict, 0.4, label = 'RF_Phase_2 Accuracy', color = '#FFC0CB')
plt.bar(X_axis + 0.2, New_predict, 0.4, label = 'TMV Accuracy', color = '#ADD8E6')
plt.plot(X_axis, percentage_left_subjects, marker='D', label = 'Data Remained', color = "#0343DF")

handles, labels = plt.gca().get_legend_handles_labels()
print(labels)
order=[2,1,0]
plt.xticks(X_axis, subject_id_order)
plt.xlabel('Subject ID orderd by RandomForest Phase 1', fontsize=15)
plt.ylabel('Accuracy and Data Remained', fontsize=15)
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], fontsize=10, loc=0) 
plt.savefig("Time_continuity_results/"+data_source+"/Accuracy_all_subjects.jpg", bbox_inches='tight', dpi = 1500)
plt.show()

In [ ]:
TMV_avg_runtime = sum(time_data)/len(time_data)
print("TMV_avg_runtime",TMV_avg_runtime)
print()
plt.figure(figsize=(10, 5))
fig, ax1 = plt.subplots()
known = []
for i in subject_id_order:
    known.append(subject_unknown_percentage[str(i)]["known"])
print("known percentage", known)
ax2 = ax1.twinx()
lns1 = ax1.plot(X_axis, known, marker='D', label = 'Train Data Remained', color = "#0343DF")
lns2 = ax2.plot(X_axis, time_data, marker='D', label = 'Runtime', color='#FF796C')
ax1.set_ylabel('Data Remained Percentage', fontsize=15)
ax2.set_ylabel('Time(s)', fontsize=15)

lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=0)
ax1.set_xlabel('Subject ID orderd by RandomForest Phase 1', fontsize=15)
plt.xticks(X_axis, subject_id_order)
plt.savefig("Time_continuity_results/"+data_source+"/Data_Time.jpg", bbox_inches='tight', dpi = 1500)
plt.show()

In [ ]:
time_spent_all_subjects = []
id_order_temp = subject_id_order[::-1]
for num in id_order_temp:
    time_spent_all_subjects.append(time_continuity_subject[num])
time_spent_all_subjects = [round(i,3) for i in time_spent_all_subjects]
data_time_elapsed = {"Code Runtime (s)": time_spent_all_subjects}
df = pd.DataFrame(data_time_elapsed, index = id_order_temp)
df.to_csv("Time_continuity_results/"+data_source+"/Runtime_all_subjects.csv")
df

In [ ]:
percentage_left_subjects = [round(j,3) for j in percentage_left_subjects]
data_percentage_left = {"Percent of data left %": percentage_left_subjects}
df_data = pd.DataFrame(data_percentage_left, index = id_order_temp)
df_data.to_csv("Time_continuity_results/"+data_source+"/Data_percentage_left_all_subjects.csv")
df_data

In [ ]:
print(len(tmv_result_dict))
for key in tmv_result_dict:
    print(key)
    print(len(tmv_result_dict[key]))
# print(tmv_result_dict["1"])

In [ ]:
#subject_1_task_1 (or the subject_id_end)
subject_id_tmv = subject_id_end - 1
interval_session_lst = [(0, 420), (420, 840), (840, 1260), (1260, 1680), (1680, 2100), (2100, 2520)]
task_num = 1
import matplotlib

tmv_predicted = tmv_result_dict[str(task_num)]
x_axis = [i + 1 for i in range(420)]
x_axis = [i/10 for i in x_axis]
# print("y_axis is", y_axis)
fig, axs = plt.subplots(6, sharex=True, sharey=True, figsize=(25, 18))


y_temp_tmv = tmv_predicted[0 : 420]
axs[0].plot(x_axis, y_temp_tmv, label = 'Time Majority Voting')
axs[0].set_yticklabels(labels=np.arange(-1,6,1), fontsize=15)
axs[0].set_title("session 1", fontsize=20)

for i in range(1, len(interval_session_lst)):
#     print(i)
    interval_session = interval_session_lst[i]
#     print(interval_session)
    y_temp_tmv = tmv_predicted[interval_session[0] : interval_session[1]]
#     print(y_temp_tmv)
#     print("="*20)
    axs[i].plot(x_axis, y_temp_tmv)
    # axs[1].legend(loc="upper right")
    axs[i].set_yticklabels(labels=np.arange(-1,6,1), fontsize=15)
    axs[i].set_title("session "+str(i + 1), fontsize=20)

plt.xticks(np.arange(0, 43, 2),fontsize=20)
plt.yticks(np.arange(-1,6,1), fontsize=15)
# plt.xticks(fontsize=15)
# matplotlib.rc('ytick', fontsize=15) 
plt.xlabel('Time Span of Task One for Each Session(s)', fontsize=26)
fig.legend(fontsize=23, loc=9)
plt.savefig("Time_continuity_results/"+data_source+"/TMV/subject_"+str(subject_id_tmv)+"_task_1.jpg", dpi = 1000) ## Or it's subject_id_end

In [ ]:
# draw heatmap
print("the length of the tmv_result_dict is", len(tmv_result_dict))
for key in tmv_result_dict:
    print(type(key))
    print(key)

In [ ]:

heatmap_data = np.zeros((5, 5))
for key in tmv_result_dict:
    res = []
    y = tmv_result_dict[key]
    y = [z for z in y if z != -1]
    for j in range(1,6):
        total_j = len([i for i in y if i == j])
        val = total_j / float(len(y))
        res.append(float(val))
    heatmap_data[int(key) - 1] = res
print(heatmap_data)
x_axis_labels = ["T1","T2","T3","T4","T5"]
y_axis_labels = ["T1:Think","T2:Count","T3:Recall","T4:Breathe","T5:Draw"]
ax = sns.heatmap(heatmap_data, cmap="Blues", vmin= -1, vmax=1, annot=True, fmt=".2f", xticklabels=x_axis_labels, yticklabels=y_axis_labels)
plt.savefig("Time_continuity_results/"+data_source+"/subejct_"+str(subject_id_tmv)+"_TMV_heatmap.jpg", dpi = 800)
plt.title("subject_" + str(subject_id_tmv) + " all six sessions TMV")